
# Dependencies

In [1]:
import custom_models
#python packages
from PIL import Image
from tqdm.notebook import tqdm
#from tqdm import tqdm
import gc
import datetime
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from skimage import io
#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
#torchvision
import torchvision
from torchvision import datasets, models, transforms
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version:  1.2.0
Torchvision Version:  0.4.0a0+6b959ee
Using GPU!


# Initialize CNN model

In [2]:
def make_CNN(model_name, num_classes, resume_from = None):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    # The model (nn.Module) to return
    model_ft = None
    # The input image is expected to be (input_size, input_size)
    input_size = 0
    
    # You may NOT use pretrained models!! 
    use_pretrained = False
    
    # By default, all parameters will be trained (useful when you're starting from scratch)
    # Within this function you can set .requires_grad = False for various parameters, if you
    # don't want to learn them

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg13":
        """ VGG13_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg16":
        """ VGG16_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg19":
        """ VGG19_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    else:
        raise Exception("Invalid model name!")
    
    return model_ft, input_size

# Define Multimodal Model

In [3]:
class Chest_Disease_Net(nn.Module):
    """
    fc1: number of neurons in the hidden fully connected layer
    """
    def __init__(self, cnn_model_name, num_classes, num_multimodal_features=12, fc1_out=32, resume_from=None):
        #num_classes = 14
        #num_multimodal_features= 12
        super(Chest_Disease_Net, self).__init__()
        self.cnn, self.input_size = make_CNN(cnn_model_name, num_classes)#models.vgg11(pretrained=False, progress = True)
        #define output layers
        self.fc1 = nn.Linear(num_classes + num_multimodal_features, fc1_out) #takes in input of CNN and multimodal input
        self.fc2 = nn.Linear(fc1_out, num_classes)
        if resume_from is not None:
            print("Loading weights from %s" % resume_from)
            self.load_state_dict(torch.load(resume_from))
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        #print("x1", x1.shape)
        x2 = data
        #print("x2", x2.shape)
        #print("x1: ", x1, type(x1))
        #print("x2: ", x2, type(x2))
        #x = torch.cat((x1, x2), dim=1)  
        x = torch.cat((x1.float(), x2.float()), dim=1) ### ???
        #print("concat", x.shape)
        x = F.relu(self.fc1(x))
        #print("relu", x.shape)
        x = self.fc2(x)
        print('forward output: ', x)
       # print("fc2", x.shape)
        return x.double() ### ???

# Data Loading

In [4]:
class MultimodalDataset(Dataset):
    """
    Custom dataset definition
    """
    def __init__(self, csv_path, img_path, transform=None):
        """
        """
        self.df = pd.read_csv(csv_path)
        self.img_path = img_path
        self.transform = transform
        self.diseases = self.get_diseases()
            
    def __getitem__(self, index):
        """
        """
        img_name = self.df.iloc[index]["img_name"] 
        img_path = os.path.join(self.img_path, img_name)
        image = Image.open(img_path)
        image = image.convert("RGB")
        image = np.asarray(image)
        if self.transform is not None:
            image = self.transform(image)
        dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor # ???
        features = np.fromstring(self.df.iloc[index]["feature"][1:-1], sep=",") # ???
        features = torch.from_numpy(features.astype("float")) # ???
        #label = int(self.df.iloc[index]['label'])
        labels = torch.tensor(list(self.df.iloc[index][self.diseases]), dtype = torch.float64)
        #print("Label type: ", type(label))
        #label = np.int_(label) #???
        #print("label type post casting: ", type(label))
        return image, features, labels
        
    def __len__(self):
        return len(self.df)
    
    def get_diseases(self):
        cols = list(self.df.columns)
        cols.remove('disease')
        cols.remove('feature')
        cols.remove('img_name')
        cols.remove('dataset_type')
        return cols
        

In [5]:
def get_dataloaders(input_size, batch_size, num_classes, augment=False, shuffle = True):
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # For now, we resize/crop the image to the correct input size for our network,
    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    data_transforms = {
        'train': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            #Add extra transformations for data augmentation
            transforms.RandomApply([
                transforms.RandomChoice([
                    transforms.RandomAffine(degrees=20),
                    transforms.RandomAffine(degrees=0,scale=(0.1, 0.15)),
                    transforms.RandomAffine(degrees=0,translate=(0.2,0.2)),
                    #transforms.RandomAffine(degrees=0,shear=0.15),
                    transforms.RandomHorizontalFlip(p=1.0)
                ] if augment else [transforms.RandomAffine(degrees=0)])#else do nothing
            ], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        'test': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ])
    }
    # Create training and validation datasets
    data_subsets = {x: MultimodalDataset(csv_path="./data/"+x+"_dataset{}.csv".format(num_classes), 
                                         img_path="/storage/images", 
                                         transform=data_transforms[x]) for x in data_transforms.keys()}
    # Create training and validation dataloaders
    # Never shuffle the test set
    dataloaders_dict = {x: DataLoader(data_subsets[x], batch_size=batch_size, shuffle=False if x != 'train' else shuffle, num_workers=4) for x in data_transforms.keys()}
    return dataloaders_dict

# Training

In [6]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, model_name=str(datetime.datetime.now()), 
                save_dir = None, save_all_epochs=False, num_epochs=25):
    '''
    model: The NN to train
    dataloaders: A dictionary containing at least the keys 
                 'train','val' that maps to Pytorch data loaders for the dataset
    criterion: The Loss function
    optimizer: The algorithm to update weights 
               (Variations on gradient descent)
    num_epochs: How many epochs to train for
    save_dir: Where to save the best model weights that are found, 
              as they are found. Will save to save_dir/weights_best.pt
              Using None will not write anything to disk
    save_all_epochs: Whether to save weights for ALL epochs, not just the best
                     validation error epoch. Will save to save_dir/weights_e{#}.pt
    '''
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # TQDM has nice progress bars
            for inputs, features, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                features = features.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs, features)
                    print("model outputs: ", outputs, outputs.size())
                    #print("model labels: ", labels.size())
                    loss = criterion(outputs, labels)

                    # torch.max outputs the maximum value, and its index
                    # Since the input is batched, we take the max along axis 1
                    # (the meaningful outputs)
                    #print("outputs: ", outputs)
                    #preds = torch.max(outputs, 1)
                    preds = (outputs > 0).type(torch.float64)
                    #_, preds = torch.max(outputs, 1)
                    #print("new preds: ", preds)
                    # backprop + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                #print("loss: ", loss.item())
                #print("inputs: ", inputs.size(0), inputs.size())
                
                running_loss += loss.item() * inputs.size(0)
                #print("running loss: ", running_loss)
                print("predictions: ", preds, preds.size())
                print("Labels: ", labels.data, labels.data.size())
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            #is the accuracy calculated correctly?
            print("running_corrects: ", running_corrects.double(), running_corrects.size())
            print("dataloaders len: ", len(dataloaders[phase].dataset)) 
            #maybe dataloaders length * number of classes? Model must predict for all classes
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), save_dir + "/{}_best_weights_1.pt".format(model_name))
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        print()
        scheduler.step()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

# Optimizer & Loss

In [7]:
def make_optimizer(model):
    # Get all the parameters
    params_to_update = model.parameters()
    print("Params to learn:")
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

    # Use SGD
    optimizer = optim.SGD(params_to_update, lr=0.01, momentum=0.9)
    return optimizer

def get_loss(num_classes,device):
    # Create an instance of the loss function
    #set weights to account for unbalanced data.
    #In expectation every category class contributes the same to the loss
    pos_weight = np.array([    0.025944895136267, 0.059170436277992,0.049543908183484,
                            0.124453250588807,0.013364985606939,0.185904145949381,
                            0.108022729821676,0.564067815619275,0.054011364910838])
    pos_weight = torch.tensor(pos_weight,dtype=torch.float64) if num_classes == 9 \
    else torch.tensor([])
    pos_weight = pos_weight.to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    return criterion

# Define Experiment Parameters

In [8]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
# You can add your own, or modify these however you wish!
model_name = "alexnet"

# Number of classes in the dataset
# Miniplaces has 100
num_classes = 9# set to 9 or 14

# Batch size for training (change depending on how much memory you have)
# You should use a power of 2.
batch_size = 64

# Shuffle the input data?
shuffle_datasets = True

# Number of epochs to train for 
num_epochs = 10

### IO
# Path to a model file to use to start weights at
#resume_from = "/home/ubuntu/6.867-xray-project/weights/data_aug_vgg.pt"
resume_from = None

# Directory to save weights to
save_dir = "weights"
os.makedirs(save_dir, exist_ok=True)

# If True saves the weights for all epochs, else only saves the weight of best one
save_all_epochs = False



# Train!

In [9]:
gc.collect()

7

In [10]:
# Initialize the model for this run

model = Chest_Disease_Net(cnn_model_name = model_name, num_classes = num_classes, resume_from = resume_from)
input_size = model.input_size
dataloaders = get_dataloaders(input_size, batch_size, num_classes, shuffle_datasets)
criterion = get_loss(num_classes=num_classes,device=device)

# Move the model to the gpu if needed
model = model.to(device)

optimizer = make_optimizer(model)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,10],gamma=0.1)
# Train the model!
trained_model, validation_history = train_model(model=model, dataloaders=dataloaders, criterion=criterion, optimizer=optimizer,
            scheduler=scheduler, model_name=model_name, save_dir=save_dir, save_all_epochs=save_all_epochs, num_epochs=num_epochs)

Params to learn:
	 cnn.features.0.weight
	 cnn.features.0.bias
	 cnn.features.3.weight
	 cnn.features.3.bias
	 cnn.features.6.weight
	 cnn.features.6.bias
	 cnn.features.8.weight
	 cnn.features.8.bias
	 cnn.features.10.weight
	 cnn.features.10.bias
	 cnn.classifier.1.weight
	 cnn.classifier.1.bias
	 cnn.classifier.4.weight
	 cnn.classifier.4.bias
	 cnn.classifier.6.weight
	 cnn.classifier.6.bias
	 fc1.weight
	 fc1.bias
	 fc2.weight
	 fc2.bias
Epoch 0/9
----------


forward output:  tensor([[ 0.1709,  0.0773,  0.1751,  0.0766,  0.2944,  0.1314,  0.1272, -0.0021,
          0.0619],
        [ 0.0897,  0.0894,  0.1869,  0.2028,  0.2433,  0.1012,  0.1654, -0.0467,
          0.1009],
        [ 0.1716,  0.0778,  0.1744,  0.0761,  0.2951,  0.1315,  0.1271, -0.0015,
          0.0619],
        [ 0.1253,  0.0733,  0.1479,  0.1797,  0.2802,  0.1728,  0.1598, -0.0167,
          0.0840],
        [ 0.1365,  0.0036,  0.1522,  0.0838,  0.3065,  0.1632,  0.1544,  0.0536,
          0.0463],
        [ 0.1711,  0.0752,  0.1745,  0.0755,  0.2940,  0.1308,  0.1262, -0.0034,
          0.0615],
        [ 0.1604,  0.0640,  0.1267,  0.1059,  0.2131,  0.1181,  0.1653, -0.0437,
          0.0701],
        [ 0.1573,  0.0368,  0.1162,  0.0934,  0.2960,  0.1355,  0.1369,  0.0532,
          0.1108],
        [ 0.1598,  0.1212,  0.1715,  0.1460,  0.3004,  0.1461,  0.1944,  0.0333,
          0.0541],
        [ 0.1709,  0.0769,  0.1752,  0.0763,  0.2945,  0.1319,  0.1277, -0.0027,
  

model outputs:  tensor([[ 0.1246,  0.0742,  0.1485,  0.1795,  0.2793,  0.1709,  0.1584, -0.0169,
          0.0817],
        [ 0.1592,  0.1201,  0.1707,  0.1451,  0.2991,  0.1457,  0.1933,  0.0325,
          0.0529],
        [ 0.1237,  0.0731,  0.1475,  0.1780,  0.2786,  0.1720,  0.1598, -0.0190,
          0.0825],
        [ 0.1596,  0.0625,  0.1265,  0.1039,  0.2130,  0.1176,  0.1655, -0.0440,
          0.0684],
        [ 0.2338,  0.0297,  0.1561,  0.0213,  0.2843,  0.1058,  0.1321, -0.0431,
          0.0265],
        [ 0.1607,  0.0621,  0.1268,  0.1038,  0.2129,  0.1180,  0.1648, -0.0447,
          0.0678],
        [ 0.0783,  0.0944,  0.1389,  0.0702,  0.1867,  0.1271,  0.1831, -0.0495,
          0.1104],
        [ 0.1568,  0.0378,  0.1143,  0.0939,  0.2941,  0.1338,  0.1350,  0.0541,
          0.1112],
        [ 0.1489,  0.1144,  0.1580,  0.1765,  0.2958,  0.1409,  0.1908,  0.0508,
          0.1191],
        [ 0.1596,  0.0630,  0.1259,  0.1047,  0.2130,  0.1181,  0.1647, -0.0442,
   

predictions:  tensor([[1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1.,

forward output:  tensor([[ 0.1423,  0.1056,  0.1514,  0.1687,  0.2875,  0.1363,  0.1858,  0.0483,
          0.1125],
        [ 0.1647,  0.0665,  0.1671,  0.0667,  0.2845,  0.1254,  0.1190, -0.0067,
          0.0531],
        [ 0.1681,  0.0207,  0.1306,  0.0867,  0.2296,  0.1492,  0.1678, -0.0716,
          0.0536],
        [ 0.1171,  0.0648,  0.1404,  0.1714,  0.2700,  0.1660,  0.1523, -0.0183,
          0.0766],
        [ 0.1680,  0.0211,  0.1297,  0.0867,  0.2297,  0.1488,  0.1668, -0.0719,
          0.0538],
        [ 0.1170,  0.0650,  0.1405,  0.1715,  0.2698,  0.1664,  0.1524, -0.0189,
          0.0763],
        [ 0.1506,  0.0280,  0.1075,  0.0849,  0.2861,  0.1283,  0.1293,  0.0519,
          0.1041],
        [ 0.1538,  0.0552,  0.1186,  0.0985,  0.2054,  0.1130,  0.1577, -0.0445,
          0.0644],
        [ 0.1500,  0.0252,  0.1077,  0.0834,  0.2857,  0.1284,  0.1302,  0.0510,
          0.1036],
        [ 0.1171,  0.0653,  0.1396,  0.1724,  0.2697,  0.1672,  0.1514, -0.0186,
  

...

Labels:  tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 

forward output:  tensor([[-0.6627, -0.8410, -0.6523, -0.8320, -0.6858, -0.6602, -0.5889, -0.2270,
         -0.7865],
        [-0.8366, -1.0523, -0.8447, -0.8250, -0.7541, -0.6948, -0.6925, -0.1444,
         -0.8197],
        [-0.8366, -0.9035, -0.7106, -0.7200, -0.7462, -0.7174, -0.6471, -0.1893,
         -0.7817],
        [-0.7575, -0.9319, -0.7377, -0.8328, -0.7096, -0.6982, -0.6417, -0.1384,
         -0.8136],
        [-0.7636, -0.9470, -0.7501, -0.8484, -0.7222, -0.7127, -0.6548, -0.1406,
         -0.8155],
        [-0.7734, -1.0493, -0.8508, -0.9413, -0.7638, -0.7343, -0.7602, -0.1444,
         -0.8518],
        [-0.8306, -1.0451, -0.8401, -0.8147, -0.7510, -0.6896, -0.6880, -0.1414,
         -0.8156],
        [-0.8372, -0.9026, -0.7107, -0.7185, -0.7461, -0.7162, -0.6463, -0.1901,
         -0.7816],
        [-0.7457, -0.9848, -0.7250, -0.9086, -0.7147, -0.5798, -0.6498, -0.1756,
         -0.8406],
        [-0.6656, -0.8462, -0.6561, -0.8370, -0.6889, -0.6636, -0.5919, -0.2281,
  

forward output:  tensor([[-0.7960, -1.1165, -0.8870, -0.9333, -0.8137, -0.7323, -0.7282, -0.1896,
         -0.8822],
        [-0.7432, -0.9789, -0.7286, -0.8984, -0.7144, -0.5798, -0.6492, -0.1648,
         -0.8503],
        [-0.6757, -0.8525, -0.6644, -0.8450, -0.6988, -0.6704, -0.6019, -0.2298,
         -0.8010],
        [-0.7220, -0.9333, -0.7450, -0.8072, -0.7381, -0.6737, -0.6508, -0.1692,
         -0.7827],
        [-0.7724, -1.0736, -0.8294, -0.9453, -0.7524, -0.7515, -0.7968, -0.1264,
         -0.8758],
        [-0.8121, -1.1212, -0.8963, -0.9578, -0.7552, -0.7320, -0.7837, -0.0787,
         -0.8381],
        [-0.7620, -1.0028, -0.7389, -0.9274, -0.7278, -0.5953, -0.6620, -0.1770,
         -0.8519],
        [-0.8424, -1.0558, -0.8506, -0.8265, -0.7625, -0.7005, -0.6987, -0.1426,
         -0.8292],
        [-0.7827, -1.0924, -0.8356, -0.9596, -0.7647, -0.7606, -0.8006, -0.1329,
         -0.8894],
        [-0.7809, -1.0592, -0.8581, -0.9489, -0.7733, -0.7414, -0.7664, -0.1454,
  

forward output:  tensor([[-0.7371, -0.9564, -0.7617, -0.8289, -0.7565, -0.6929, -0.6674, -0.1724,
         -0.7981],
        [-0.7319, -1.0321, -0.8137, -0.9574, -0.7183, -0.7273, -0.7072, -0.1386,
         -0.8838],
        [-0.8591, -1.0788, -0.8680, -0.8475, -0.7785, -0.7171, -0.7137, -0.1476,
         -0.8429],
        [-0.7948, -0.9643, -0.7470, -0.8060, -0.7212, -0.7085, -0.6666, -0.1290,
         -0.8104],
        [-0.8538, -1.0689, -0.8623, -0.8377, -0.7735, -0.7122, -0.7087, -0.1424,
         -0.8402],
        [-0.6972, -0.8865, -0.6912, -0.8754, -0.7241, -0.6991, -0.6251, -0.2317,
         -0.8263],
        [-0.8061, -1.1232, -0.8933, -0.9401, -0.8211, -0.7386, -0.7338, -0.1898,
         -0.8901],
        [-0.7974, -1.0758, -0.8739, -0.9657, -0.7880, -0.7573, -0.7826, -0.1467,
         -0.8776],
        [-0.7357, -0.9573, -0.7640, -0.8276, -0.7571, -0.6927, -0.6697, -0.1706,
         -0.7955],
        [-0.7934, -1.1058, -0.8554, -0.9689, -0.7777, -0.7759, -0.8210, -0.1300,
  

KeyboardInterrupt: 